In [1]:
import numpy
import math
import pandas as pd

### Advanced Atmospheric Blackbody Temperature of Earth

CSV containing specified wavelengths and corresponding absorption spectrum data

In [2]:
df = pd.read_csv('absorbtionspectra.csv')
w = df.w
ab = df.ab

Constants


In [3]:
# Planck's constant
h = 6.626*(10**-34)
# speed of light
c = 2.9979*(10**8)
# Boltzmann's constant
K = 1.381*(10**-23)
A = 2*h*c*c
B = h*c/K
# mass of Earth
m = 5.97*(10**24)
# radius of Earth
Re = 6378*1000
# distance from Earth to sun
d = 149600000000
# radius of sun
rs = 696000000
# number of r's away from source
D = (d/rs)**2

# temperature of the Sun (K)
Ts = 5772
# temperatue of space (2)
Te = 2

Correction based on planetary albedo (~0.3) and non-directional scattering (assuming an equal probability of atmospheric particles re-emitting waves in any direction)

In [4]:
# 1*abs[i] is absorbed
# 0.7*trans[i] is transmitted
# 0.3*trans[i] is transmitted and then reflected off Earth's surface
    # 0.3*trans[i]*trans[i] is reflected back out
    # 0.3*trans[i]*abs[i] is reabsorbed
# Total trans = 0.7*trans[i] + 0.3*trans[i]*trans[i]
# Total abs = 1*abs[i] + 0.3*trans[i]*abs[i]

Temperature of Earth

In [7]:
dt = 10

while abs(dt) > 0.00001:
   # creating multiple arrays to sum
    SRtransdirect = []
    SRabsdirect = []
    SRabsreflect = []
    for i in range (len(w)-1):
        SRtransnext = ab[i]*A/((w[i]**5)*((math.e**(B/(w[i]*Ts))-1)))
        SRabsnext = (1-ab[i])*A/((w[i]**5)*((math.e**(B/(w[i]*Ts))-1)))
        dw = w[i+1]-w[i]
        SRtransdirect.append(0.7*SRtransnext*dw)
        SRabsdirect.append(SRabsnext*dw)
        SRabsreflect.append(0.3*SRtransnext*(1-ab[i])*dw)
    TRtrans = sum(SRtransdirect)/D
    TRabs = sum(SRabsdirect)/D + sum(SRabsreflect)/D
    Ein = TRtrans*math.pi*Re*Re + 0.5*TRabs*math.pi*Re*Re
    SRout = []
    SRoutabs = []
    for j in range (len(w)-1):
        SRoutnext = ab[j]*A/((w[j]**5)*((math.e**(B/(w[j]*Te))-1)))
        SRoutabsnext = (1-ab[j])*A/((w[j]**5)*((math.e**(B/(w[j]*Te))-1)))
        dw = w[j+1]-w[j]
        SRout.append(SRoutnext*dw)
        SRoutabs.append(SRoutabsnext*dw)
    TRout = sum(SRout) + 0.5*sum(SRoutabs)
    Eout = TRout*4*math.pi*Re*Re
    dE = Ein - Eout
    dt = dE/(10**16)
    Te = Te + dt
print("Temperature of Earth", Te, "K")

/var/folders/p3/678x5jr9329c83n3fr8gv_240000gp/T/ipykernel_94667/3483613221.py:21: RuntimeWarning: overflow encountered in double_scalars
  SRoutnext = ab[j]*A/((w[j]**5)*((math.e**(B/(w[j]*Te))-1)))
/var/folders/p3/678x5jr9329c83n3fr8gv_240000gp/T/ipykernel_94667/3483613221.py:22: RuntimeWarning: overflow encountered in double_scalars
  SRoutabsnext = (1-ab[j])*A/((w[j]**5)*((math.e**(B/(w[j]*Te))-1)))


Temperature of Earth 282.33908738513327 K
